## Короткая шпаргалка по базовым метрикам классификации

Классифицировать будем вручную. Цель - посмотреть как считаются и что показывают метрики. Так же, попробуем написать свои функции для рассчета метрик.

У нас будет 2 списка из 10 значений: "истинный" и "предстказанный":

In [1]:
import numpy as np
import pandas as pd

In [2]:
np.random.seed(0)
true = np.random.randint(0, 2, 10)
pred = np.random.randint(0, 2, 10)

print('true:', true)
print('pred:', pred)

true: [0 1 1 0 1 1 1 1 1 1]
pred: [1 0 0 1 0 0 0 0 0 1]


### Confusion matrix

In [3]:
from sklearn.metrics import confusion_matrix

Это, наверное, основная концепция, которую стоит понять, чтобы хорошо разбираться в метриках, построенных на отношении количества ответов. В примере у нас есть два класса и алгоритм (random 😀), предсказывающий принадлежность каждого объекта одному из классов. Как должна выглядеть матрица ошибок правильно, я не знаю до сих пор 😅, но, чтобы понимать, что стоит за матрицей, которая строится силами sklearn, буду показывать на примере расположения ее осей:

![title](./images/01.png)

Строки матрицы соответствуют истинным значениям, столбцы - предсказанным. Когда алгоритм относит объект к первому классу, говорим, что он - **срабатывает**. А когда к нулевому классу - **пропускает**. Всвязи с этим, у нас могут возникнуть 4 ситуации:

- Алгоритм сработал на объект и предсказал **1**, при этом, истинная метка, тоже **1**. Получаем верное срабатывание **TruePositive** (**TP**).
- Алгоритм сработал на объект и предсказал **1**, при этом, истинная метка - **0**. Получаем ложное срабатывание **FalsePositive** (**FP**).
- Алгоритм пропустил объект и предсказал **0**, при этом, истинная метка - **1**. Получаем ложный пропуск **FalseNegative** (**FN**).
- Алгоритм пропустил объект и предсказал **0**, при этом, истинная метка, тоже **0**. Получаем верный пропуск **TrueNegative** (**TN**).

Таким образом, сумма по каждому столбцу отражает предсказанное количество классов, а сумма по каждой строке - фактическое.

Если мы сейчас соберем наши значения, то получим такую картину:

![title](./images/02.png)

Логика - простая:

In [4]:
print('true:', true)
print('pred:', pred)

true: [0 1 1 0 1 1 1 1 1 1]
pred: [1 0 0 1 0 0 0 0 0 1]


- Первая пара: ложный пропуск - **FN**
- Вторая пара: верное срабатывание - **TP**
- Третья пара: ложное срабатывание - **FP**<br>
...<br>
- Пятая_ пара: верный пропуск - **TN**

Остальное можно разложить в клетки самостоятельно.

Ну и, естественно, нет необходимости, каждый раз это все делать вручную. В бибилиотеке sklearn, как я писал выше, есть готовый метод: **confusion_matrix**

In [5]:
confusion_matrix(true, pred)

array([[0, 2],
       [7, 1]])

In [6]:
TN, FP, FN, TP = confusion_matrix(true, pred).ravel()

Как видим, наша матрица и sklearn'овская - совпали. Теперь мы видим, что 5 единичек были приняты нашим алгоритмом за нули, и 1 ноль был принят за единичку. На этой базе и будем считать дальнейшие метрики.

### Accuracy (доля верных ответов)

In [7]:
from sklearn.metrics import accuracy_score

Наверное, наиболее интуитивно понятная метрика:

$\displaystyle Accuracy = \frac{TP + TN}{TN + FN + TP + FP}$

(отношение количества верных ответов, по всем классам, к общему количеству объектов)

Чуть усложним картинку, чтобы понять верно:

![title](./images/03.png)

К сожалению - бесполезна при дисбалансе классов. Например, в нашем случае:

In [8]:
print('Ручной расчет:', (TP + TN) / (TN + FN + TP + FP))
print('accuracy_score:', accuracy_score(true, pred))
print('Dummie predict:', accuracy_score(true, np.ones(10)))

Ручной расчет: 0.1
accuracy_score: 0.1
Dummie predict: 0.8


Т.е., если будем сравнивать модели только по этой метрике, та, что предсказана "дурацким алгоритмом" - выглядит лучше. Чтобы такого не происходило, стоит смотреть на более продвинутые метрики.

### Precision (точность)

In [9]:
from sklearn.metrics import precision_score

**Precision** - дает представление о модели с точки зрения ее ложно-положительного срабатывания. Т.е. показывает долю истинных срабатываний от их общего количества.

Думаю, формула и картинка - точнее передадут мою мысль:

$\displaystyle Precision = \frac{TP}{TP + FP}$

Если precision на наших данных и сравнить с библиотечной реализацией, получится так:

In [10]:
print('Ручной расчет:', TP / (TP + FP))
print('Precision_score:', precision_score(true, pred))
print('Dummie predict:', precision_score(true, np.ones(10)))

Ручной расчет: 0.3333333333333333
Precision_score: 0.3333333333333333
Dummie predict: 0.8


## Recall (полнота)

In [11]:
from sklearn.metrics import recall_score

Можно сказать, что **Recall** - это обратная метрика от Precision (она минимизирует ложные пропуски). Т.е. - показывает долю верно угаданных объектов положительного класса, среди всех объектов положительного класса.

$\displaystyle Recall = \frac{TP}{TP + FN}$

In [12]:
print('Ручной расчет:', TP / (TP + FN))
print('Recall_score:', recall_score(true, pred))
print('Dummie predict:', recall_score(true, np.ones(10)))

Ручной расчет: 0.125
Recall_score: 0.125
Dummie predict: 1.0


Две эти метрики (*Recall* и *Precision*) тесно связаны, и, зачастую, если отбросить идеальные случаи, при росте одного показателя - портится другой. А значит, ставя задачу и выбирая метрику, нужно понимать, что нам важнее минимизировать - ложные срабатывания или ложные пропуски. 

### F1_score

In [13]:
from sklearn.metrics import f1_score



есть ничто иное, как гармоническое среднее между **Precision** и **Recall**.

$\displaystyle F1 = \frac{2}{\frac{1}{Precision} \cdot \frac{1}{Recall}} = 2 \cdot \frac{Precision \cdot Recall}{Precision + Recall}$

Или, если оперировать только теми показателями, которые у нас есть:

$\displaystyle F1 = \frac{TP}{TP + \frac{FN + FP}{2}}$

In [14]:
print('Ручной расчет:', TP / (TP + ((FN + FP) / 2)))
print('Recall_score:', f1_score(true, pred))
print('Dummie predict:', f1_score(true, np.ones(10)))

Ручной расчет: 0.18181818181818182
Recall_score: 0.18181818181818182
Dummie predict: 0.888888888888889


Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, а recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.


https://towardsdatascience.com/understanding-the-confusion-matrix-from-scikit-learn-c51d88929c79

https://habr.com/ru/company/ods/blog/544208/